##### This notebook will attempt to add the specified security group OID to all workspaces.
###### Due to limits on the API it will assign the security group to 200 workspaces at a time and then wait one hour.
###### It will run on a schedule every hour until all workspaces (not in the ignore list parameter below) have had the security group assigned.   

Prerequisites: 
- Only tenant admin can run this notebook 
- Ensure a default lakehouse has been set.
- Ideally this should run in a spark environment with using a small single node pool to avoid unnecessary capacity consumption

In [ ]:
# Specify the OID of the security group
p_security_group_oid = 'dd0c31c7-462d-4324-920f-193e40516dc4'

# Specify workspaces to ignore 
# Either use exact workspace names or prefix or suffix with % as wildcard 
# This is useful when you have have any/many workspaces which you do not wish to add the security group to 
# e.g. to ignore any workspaces suffixed with DEV or TEST use ['%DEV','%TEST%'] 

# These list parameters need to be in the format of ['string1','string2',...'stringN']. Use [] for an empty list.

# Specify an exact list of workspaces to ignore e.g. p_ws_ignore_list = ['Microsoft Fabric Capacity Metrics 26/02/2024 16:15:42','AdminInsights']
p_ws_ignore_list = [] 
# Specify a list with wildcards using % e.g. to ignore anything with _DEV and _TEST as a suffix p_ws_ignore_like_list = ['%_DEV%','%_TEST%']  
p_ws_ignore_like_list = []

# Optional: specify capacity ID
p_capacity_id = ['5B83C51B-6B94-495A-8CC6-875F7F63248B']

##### Utility functions

In [ ]:
from pyspark.sql.functions import col,current_timestamp,lit
import sempy.fabric as fabric
from pyspark.sql import DataFrame
import pandas as pd

def saveTable(pdf,table_name, mode='overwrite'):
    if mode=='append' and not any(table.name == table_name for table in spark.catalog.listTables()):
            mode = 'overwrite'

    if (isinstance(pdf, pd.DataFrame) and pdf.empty) or \
       (isinstance(pdf, DataFrame) and pdf.isEmpty()):
        return('No ' + table_name + ' found, nothing to save (Dataframe is empty)')
    if not isinstance(pdf, DataFrame):
        pdf = spark.createDataFrame(pdf)

    df = pdf.select([col(c).alias(
            c.replace( '(', '')
            .replace( ')', '')
            .replace( ',', '')
            .replace( ';', '')
            .replace( '{', '')
            .replace( '}', '')
            .replace( '\n', '')
            .replace( '\t', '')
            .replace( ' ', '_')
            .replace( '.', '_')
        ) for c in pdf.columns])
    #display(df)
    df.withColumn("metaTimestamp",current_timestamp()) \
      .withColumn("admingroupassigned",lit(0)).write.mode(mode) \
      .option("mergeSchema", "true").saveAsTable(table_name)
    return(str(df.count()) +' records saved to the '+table_name + ' table.')

def saveWorkspaceMeta(suppress_output=False):
    spark.sql("drop table if exists workspaces")
    df = fabric.list_workspaces()
    #display(df)
    if not suppress_output:
        print(saveTable(df,"workspaces"))
    else:
        saveTable(df,"workspaces")



##### Check whether a schedule has been defined
This is done so that when the notebook is run the first time, it will set a flag to create the schedule
Any subsequent runs it will not attempt to create the schedule but will store the schedule ID for when the job ends it can be disabled.

In [ ]:
from datetime import datetime, timedelta
import sempy.fabric as fabric
import pytz

create_schedule = False
#check if schedule is already enabled
client = fabric.FabricRestClient() 
url  = f'v1/workspaces/{notebookutils.runtime.context["currentWorkspaceId"]}/items/{notebookutils.runtime.context["currentNotebookId"]}/jobs/RunNotebook/schedules'
response = client.get(url)
#print(response.text)
if not response.json()["value"]:
    print('No schedule defined yet')
    create_schedule=True
else:
    current_schedule_id = response.json()["value"][0]['id']
    print('Current schedule ID '+ current_schedule_id)



##### If no schedule exists, create one to start in 10 minutes with a 60 minute interval. 
This typically happens on the first run. Any modifications to the schedule thereafter will need to be manually done using the run tab in the menu bar

Additionally store a list of workspaces in a lakehouse table to control the batches of workspaces in each run 

In [ ]:

if create_schedule:
    now= datetime.now(pytz.timezone('GMT'))
    start_time = now + timedelta(minutes=10)
    formatted_start_time = start_time.strftime("%Y-%m-%dT%H:%M:%S")
    #print(formatted_start_time)
    now= datetime.now(pytz.timezone('GMT'))
    end_time = now + timedelta(days=5)
    formatted_end_time = end_time.strftime("%Y-%m-%dT%H:%M:%S")


    client = fabric.FabricRestClient() 
    payload = {
    "enabled": True,
    'configuration': {'type': 'Cron', 'startDateTime': formatted_start_time, 'endDateTime': formatted_end_time, 'localTimeZoneId': 'GMT Standard Time', 'interval': 60}
    }
    url  = f'v1/workspaces/{notebookutils.runtime.context["currentWorkspaceId"]}/items/{notebookutils.runtime.context["currentNotebookId"]}/jobs/RunNotebook/schedules'
    print(url)
    response = client.post(url,json=payload)
    if response.json()["id"]:
        current_schedule_id = response.json()["id"]
        current_schedule_config = response.json()["configuration"]
        print('Schedule ID: ' +current_schedule_id + '. Config: '+str(current_schedule_config))
    else:
        print('Schedule not created: '+str(response.status_code)+ ' '+response.text)

    # Lastly save a list of workspaces to a table in the default lakehouse. This will be used to control the batches of workspaces in each run.
    saveWorkspaceMeta()
else:
    print('Schedule already defined. Please view details in the run tab in the menu bar.')

##### Sequentially loops through 200 workspaces at a time, based on filters specified, and assigns the security group.
###### If success then the field admingroupassigned is set to 1, if error the field is set to 2

In [ ]:
print('This cell will attempt to add the specified security group OID to 200 workspaces at a time.')

wkssql  ="SELECT distinct ID,Type,Name FROM workspaces where Type!='AdminInsights' and admingroupassigned=0 "
if len(p_capacity_id)>0:
    upper_cids = [cid.upper() for cid in p_capacity_id]
    wkssql  = wkssql + " and upper(Capacity_Id) in ('" + "', '".join(upper_cids)+ "') "

if len(p_ws_ignore_like_list)>0:
    for notlike in p_ws_ignore_like_list:
        wkssql  = wkssql + " and Name not like '" + notlike + "'"
if len(p_ws_ignore_list)>0:
    wkssql  = wkssql + " and Name not in ('" + "', '".join(p_ws_ignore_list)+ "') "
wkssql = wkssql + " limit 200"
# Uncomment the line below to see the query that is generated
#print(wkssql)
dfwks = spark.sql(wkssql).collect()
print("The following workspaces will have the security specified group added. If you wish to proceed, run the cells below. If not please disable the schedule.")
display(dfwks)



In [ ]:
if len(dfwks)>0:
    counter = 0 
    payload = {"identifier": p_security_group_oid, "principalType": "Group", "groupUserAccessRight": "Admin"}
    client = fabric.PowerBIRestClient()
    print('Started at '+  str(datetime.now()))

    print("Assigning security group to " + str(len(dfwks)) + " workspaces. This may take a few minutes...")

    for idx,i in enumerate(dfwks):
        if i['Type'] == 'Workspace':
            url = f"/v1.0/myorg/admin/groups/{i['ID']}/users"
            counter = counter+1
            if counter%50==0:
                print('Group added to 50 workspaces at '+ str(datetime.now()))
            try:
                response = client.post(url,json=payload)
                # If successful then update the admingroupassigned field to 1
                spark.sql("update workspaces set admingroupassigned=1 where id ='"+i['ID']+"'")
            except Exception as error:
                errmsg =  "Couldn't add group to workspace " + i['Name'] + "("+ i['ID'] + "). Error: "+str(error).split('"code":"')[1].split('"')[0]
                print(str(errmsg))
                # If there was an error tthen update the admingroupassigned field to 2
                spark.sql("update workspaces set admingroupassigned=2 where id ='"+i['ID']+"'")
    print('Completed batch of 200 at '+  str(datetime.now()))    
else:
    print('Done - no further workspaces to assign')
    # Disabling schedule
    payload = {'enabled': False,'configuration': current_schedule_config}
    url  = f'v1/workspaces/{notebookutils.runtime.context["currentWorkspaceId"]}/items/{notebookutils.runtime.context["currentNotebookId"]}/jobs/RunNotebook/schedules/{current_schedule_id}'
    print(url)
    response = client.patch(url,json=payload)
    if response.status_code==200:
        print('Schedule disabled')
    else:
        print('Cannot disable schedule, please disable manually')


In [ ]:
df = spark.sql("SELECT admingroupassigned,count(*) FROM workspaces group by admingroupassigned")
display(df)